<a href="https://colab.research.google.com/github/shashank2123/Punctuation-Restoration-For-Youtube-Transcript/blob/main/Punctuation_Restoration_Using_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Punctuation-Restoration-Using-Hugging-Face

Though people rarely speak in complete sentences, punctuation confers many benefits to the readers of transcribed speech. Unfortunately, most ASR systems do not produce punctuated output. So, Here We are going to build Punctuation Restoration System for youtube transcript just by providing  URL of Youtube Video.

### Setup

In [1]:
!pip install -q datasets transformers  nltk
!pip install sentencepiece
!pip install youtube_transcript_api

     |████████████████████████████████| 264 kB 13.2 MB/s 
     |████████████████████████████████| 2.6 MB 43.9 MB/s 
     |████████████████████████████████| 118 kB 51.5 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 243 kB 49.5 MB/s 
     |████████████████████████████████| 895 kB 42.2 MB/s 
     |████████████████████████████████| 636 kB 44.8 MB/s 
     |████████████████████████████████| 3.3 MB 41.4 MB/s 
     |████████████████████████████████| 1.2 MB 12.4 MB/s 


In [2]:
!nvidia-smi

Tue Aug 17 14:39:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P8    36W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
import os
import re
import random
import nltk
import pandas as pd
import numpy as np
from tqdm import tqdm 
from nltk import word_tokenize
from datasets import load_dataset , load_metric
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
from youtube_transcript_api import YouTubeTranscriptApi


nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print("device :- ",device)

device :-  cuda


In [ ]:
model_name = 't5-base'

In [ ]:
!huggingface-cli login
!pip install hf-lfs
!git config --global user.email "shashankkulal7@gmail.com"
!git config --global user.name "Shashank"


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: shashank2123
Password: 
Login successful
Your token: DPRfpJDItBDrADZCaOQRbcYnvxIiJlwXwblMpPwBuqPhsXpBemzotUEkDKKWzPunIczjdyFqRmglGTzhaNmjeciiwmIPYhTxBMucKBEWqnganESlkJTPayBFHWoXaXXk 

Your token has been saved to /root/.huggingface/token
     |████████████████████████████████| 4.0 MB 7.4 MB/s 


### Dataset


We are using [TED – Ultimate Dataset](https://www.kaggle.com/miguelcorraljr/ted-ultimate-dataset) datasets.TED is devoted to spreading powerful ideas in just about any topic. These datasets contain over 4,000 TED talks including transcripts in many languages. For this task we are only using english language dataset. 

Here We remove only these `['.',',','?','"',';',':','-','!']` Punctuation marks which act as input to the model.

In [ ]:
def to_list(df_data, MAX_LEN = 200 , num = 100):
    transcript_text = df_data.transcript.values.tolist()
    
    slice_data = []
    
    for text in tqdm(transcript_text):
      
        ## spliting data by space to count number of words
        split_text = text.split(' ')

        for idx in range(0 , len(split_text) , MAX_LEN):
    
            slice_text = " ".join(split_text[idx:idx+MAX_LEN])

            ## removing above listed Punctuation marks
            ctext = re.sub('[\.,\?\";:\-!]',' ',slice_text)
            
            slice_data.append((slice_text, ctext ))
            
    return slice_data

In [ ]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Punctuation-Restoration-For-YouTube-Transcript/data/ted_talks_en.csv"
df_data = pd.read_csv(data_path)
num = 100
data_list = to_list(df_data.iloc[:-num,:] , MAX_LEN=200)
train_data = data_list[:-1000]
val_data = data_list[-1000:]
test_data = df_data.iloc[-num:,:].transcript.values.tolist()

print("\nTotal data length :- ",len(data_list))
print("Length of train sentence :- ",len(train_data))
print("Length of valid sentence :- ",len(val_data))
print("Length of test data sentence :- ",len(test_data))

100%|██████████| 3905/3905 [00:01<00:00, 3709.97it/s]


Total data length :-  37084
Length of train sentence :-  36084
Length of valid sentence :-  1000
Length of test data sentence :-  100


In [ ]:
train_data[random.randint(0,len(train_data))]

("get a disease that's going to possibly kill me 10 years from now, or I could use my own needle now and go to jail tomorrow. And while junkies think that it's a really bad idea to expose themselves to HIV, they think it's a much worse idea to spend the next year in jail where they'll probably end up in Frankie's situation and expose themselves to HIV anyway. So, suddenly it becomes perfectly rational to share needles. Now, let's look at it from a policy maker's point of view. This is a really easy problem. For once, your incentives are aligned. We've got what's rational for public health. You want people to use clean needles — and junkies want to use clean needles. So we could make this problem go away simply by making clean needles universally available and taking away the fear of arrest. Now, the first person to figure that out and do something about it on a national scale was that well-known, bleeding heart liberal Margaret Thatcher. And she put in the world's first national needle

In [ ]:
data_dir = 'data'
if not os.path.isdir(data_dir):
  os.mkdir(data_dir)

TRAIN_PATH = os.path.join(data_dir,"train.csv")
VALID_PATH = os.path.join(data_dir,"val.csv")
TEST_PATH = os.path.join(data_dir,"test.csv")

train_df = pd.DataFrame({'Without_Punct':[temp[1] for temp in train_data], 'With_Punct':[temp[0] for temp in train_data]})
train_df.replace('', np.nan, inplace=True)
train_df.dropna(inplace=True)
train_df.to_csv(TRAIN_PATH , index=False)
val_df = pd.DataFrame({'Without_Punct':[temp[1] for temp in val_data], 'With_Punct':[temp[0] for temp in val_data]})
val_df.replace('', np.nan, inplace=True)
val_df.dropna(inplace=True)
val_df.to_csv(VALID_PATH , index=False)


ModelDataset = load_dataset('csv', data_files={'train': TRAIN_PATH,
                                          'val': VALID_PATH})

Using custom data configuration default-814c07c77206a5d2


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-814c07c77206a5d2/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


In [ ]:
ModelDataset

DatasetDict({
    train: Dataset({
        features: ['Without_Punct', 'With_Punct'],
        num_rows: 36084
    })
    val: Dataset({
        features: ['Without_Punct', 'With_Punct'],
        num_rows: 1000
    })
})

#### Preparing data for the model

In [ ]:
MAX_LEN = 256

tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):

    inputs = [ ex for ex in examples['Without_Punct']]
    targets = [ex for ex in examples['With_Punct']]

    model_inputs = tokenizer(inputs,padding='max_length', truncation=True,max_length=MAX_LEN)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=MAX_LEN, padding='max_length', truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
preprocess_function(ModelDataset['train'][:2])

{'input_ids': [[1562, 25, 78, 231, 4409, 275, 34, 31, 7, 1892, 3, 9, 248, 3610, 12, 43, 8, 1004, 12, 369, 12, 48, 1726, 4394, 27, 31, 51, 2033, 7335, 27, 43, 118, 3, 17378, 550, 57, 48, 2542, 11, 27, 241, 12, 2763, 66, 13, 25, 21, 8, 186, 1245, 2622, 81, 125, 27, 141, 12, 497, 8, 119, 706, 275, 27, 497, 24, 3, 27518, 16509, 250, 41, 329, 3961, 78, 115, 61, 27, 174, 24, 41, 26604, 449, 61, 5306, 28106, 16, 82, 1102, 41, 26604, 449, 61, 27, 8079, 210, 30, 1761, 5205, 2759, 21, 2641, 203, 41, 26604, 449, 61, 852, 27, 43, 12, 240, 326, 82, 4439, 42, 10427, 12, 129, 30, 46, 20527, 41, 26604, 449, 61, 41, 9648, 521, 1074, 61, 27, 31, 195, 817, 25, 80, 1704, 733, 12, 11485, 125, 24, 31, 7, 118, 114, 21, 140, 41, 26604, 449, 61, 94, 31, 7, 3, 9, 1176, 733, 3, 318, 334, 720, 13, 48, 19, 1176, 14902, 227, 18332, 49, 11, 27, 646, 8, 3, 318, 41, 329, 3961, 78, 115, 61, 1945, 1384, 3, 318, 41, 26604, 449, 61, 62, 130, 2191, 45, 69, 234, 16, 17659, 12, 3, 9, 385, 3797, 62, 43, 943, 2286, 5727, 13, 1

In [ ]:
tokenized_datasets = ModelDataset.map(preprocess_function, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### Fine-tuning the model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.cuda()

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [ ]:
batch_size = 10
output_dir = "/content/drive/MyDrive/Colab Notebooks/Punctuation-Restoration-For-YouTube-Transcript/output"
repo_name = "t5-base-fine-tuned-for-Punctuation-Restoration"
args = Seq2SeqTrainingArguments(
    output_dir = 'punctuation_restoration_system',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    push_to_hub_model_id=repo_name,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using amp fp16 backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Without_Punct, With_Punct.
***** Running training *****
  Num examples = 36084
  Num Epochs = 3
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 10827


Epoch,Training Loss,Validation Loss
1,nan,0.072593
2,nan,0.068936


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1316: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
Saving model checkpoint to punctuation_restoration_system/checkpoint-500
Configuration saved in punctuation_restoration_system/checkpoint-500/config.json
Model weights saved in punctuation_restoration_system/checkpoint-500/pytorch_model.bin
tokenizer config file saved in punctuation_restoration_system/checkpoint-500/tokenizer_config.json
Special tokens file saved in punctuation_restoration_system/checkpoint-500/special_tokens_map.json
Copy vocab file to punctuation_restoration_system/checkpoint-500/spiece.model
Saving model checkpoint to punctuation_restoration_system/checkpoint-1000
Configura

Epoch,Training Loss,Validation Loss
1,nan,0.072593
2,nan,0.068936
3,nan,0.067617


Saving model checkpoint to punctuation_restoration_system/checkpoint-9000
Configuration saved in punctuation_restoration_system/checkpoint-9000/config.json
Model weights saved in punctuation_restoration_system/checkpoint-9000/pytorch_model.bin
tokenizer config file saved in punctuation_restoration_system/checkpoint-9000/tokenizer_config.json
Special tokens file saved in punctuation_restoration_system/checkpoint-9000/special_tokens_map.json
Copy vocab file to punctuation_restoration_system/checkpoint-9000/spiece.model
Deleting older checkpoint [punctuation_restoration_system/checkpoint-7500] due to args.save_total_limit
Saving model checkpoint to punctuation_restoration_system/checkpoint-9500
Configuration saved in punctuation_restoration_system/checkpoint-9500/config.json
Model weights saved in punctuation_restoration_system/checkpoint-9500/pytorch_model.bin
tokenizer config file saved in punctuation_restoration_system/checkpoint-9500/tokenizer_config.json
Special tokens file saved in 

TrainOutput(global_step=10827, training_loss=nan, metrics={'train_runtime': 6940.3876, 'train_samples_per_second': 15.597, 'train_steps_per_second': 1.56, 'total_flos': 6.592090956890112e+16, 'train_loss': nan, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to punctuation_restoration_system
Configuration saved in punctuation_restoration_system/config.json
Model weights saved in punctuation_restoration_system/pytorch_model.bin
tokenizer config file saved in punctuation_restoration_system/tokenizer_config.json
Special tokens file saved in punctuation_restoration_system/special_tokens_map.json
Copy vocab file to punctuation_restoration_system/spiece.model


'https://huggingface.co/shashank2123/t5-base-fine-tuned-for-Punctuation-Restoration/commit/879289ccfc4d5b5e4a980e009b6f3bb612f1cf6e'

### Model Evaluating

#### How Precisions , Recall , F1-Score are Calculated?

We used ROUGE metrics  technique but only for punctuation marks. Rouge metrics uses all the tokens for comparision. In this metrics evaluation which extractes punctuation marks and its both adjacent word tokens, then Precisions,Recall and F1-Score are calculated.

For example:-

```

candidate sentence = "Hello How are you?"

reference setence = "Hello, How are you?"

cand_grams = ["you ? </s>"]

ref_grams = ["Hello , How" , "you ? </s>" ]

common_grams = ["you ? </s>"]

```

**Recall**

The recall counts the number of overlapping n-grams found in both the model output and reference — then divides this number by the total number of n-grams in the reference. It looks like this: 

<center><img src = "https://miro.medium.com/max/3000/1*XEnhQJxKbEySimh1PPWPnQ.png" height = 300 width = 600 ></center>

For above example recall would be **0.5** that is number of `common_grams` dividing by total number of `ref_grams`.

**Precision**

precision metric — which is calculated in almost the exact same way, but rather than dividing by the reference n-gram count, we divide by the model n-gram count.

<center><img src = "https://miro.medium.com/max/3000/1*aSd89F6kupr3znW71Qmb3Q.png" height = 100 width = 600 ></center>

For above example precision would be **1** that is number of `common_grams` dividing by total number of `cand_grams`.

**F1-Score**

Now that we have both the recall and precision values, we can use them to calculate our F1 score like so :-

<center><img src = "https://miro.medium.com/max/875/1*zYuwaCDNpYf51H5S4DpDRA.png" height = 100 width = 300 ></center>

For above example F1-score would be **0.666**

In [ ]:
from nltk import word_tokenize

def return_punctuation_gram(text):
    text_tokens = word_tokenize(text)
    text_tokens = ['<s>'] + text_tokens + ['</s>']
    output = []
    i = 1
    while(i<len(text_tokens)):
        if text_tokens[i] in ['.',',','?','"',';',':','-','!']:
            temp_text = " ".join(text_tokens[i-1:i+2])
            output.append(temp_text)
        i+=1
    return output

def metrics_eval(pred_list , actual_list):

  tr_recall = 0.0
  tr_precisions = 0.0
  tr_f1_score = 0.0
  total_step = 0

  for pred , label in zip(pred_list , actual_list):

    pred_tokens = return_punctuation_gram(pred)
    label_tokens = return_punctuation_gram(label)
    ep = 1e-5
    precisions = len(set(label_tokens).intersection(pred_tokens))/(len(pred_tokens)+ep)
    recall = len(set(label_tokens).intersection(pred_tokens))/(len(label_tokens)+ep)
    f1_score = 2*precisions*recall/(precisions+recall + ep)

    tr_precisions += precisions
    tr_recall += recall
    tr_f1_score += f1_score
    total_step+=1

  result = {
      'precisions':tr_precisions/total_step,
      'recall': tr_recall/total_step,
      'F1_score': tr_f1_score/total_step,
  }

  return result

In [ ]:
def prediction(dataset , model , tokenizer,batch_size = 8):
  model.eval()
  MAX_LEN = 256
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  result = []
  batch_data = []
  for i in range(0,len(dataset), batch_size):
    batch_data.append(dataset[i:i+batch_size])

  for text in tqdm(batch_data):

    tokenized_data = tokenizer(text ,padding="max_length" , max_length=MAX_LEN , truncation = True, return_tensors='pt')

    ids = tokenized_data['input_ids'].to(device, dtype = torch.long)
    mask = tokenized_data['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
                input_ids = ids,
                max_length=256, 
                attention_mask = mask,
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    result.append(preds)
  result = [text for batch in result for text in batch]
  return result

In [ ]:
PATH = "shashank2123/t5-base-fine-tuned-for-Punctuation-Restoration"
tokenizer = AutoTokenizer.from_pretrained(PATH)
model = AutoModelForSeq2SeqLM.from_pretrained( PATH )
model.cuda()
model.eval()
test_x = [ temp[1] for temp in val_data]
test_y = [ temp[0] for temp in val_data]
test_pred = prediction(test_x , model , tokenizer , 16)
df_pred = pd.DataFrame({'actual':test_y,'pred':test_pred})
result = metrics_eval(test_pred , test_y)
print('Precisions :- ',result['precisions'],
      'recall :- ', result['recall'],
      'F1_score :- ', result['F1_score'])

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

  0%|          | 0/63 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
100%|██████████| 63/63 [08:32<00:00,  8.14s/it]


Precisions :-  0.8963644673620141 recall :-  0.793530868525502 F1_score :-  0.8384633131663003


### Evaluating with test dataset

Test dataset is not divided by MAX_LEN, It is a complete transcript of Youtube video. Now this evaluation of model tells how model perfom on real world data.

In [8]:
def predict_full_text(transcript_text , model = None , tokenizer = None):
  
  PATH = "shashank2123/t5-base-fine-tuned-for-Punctuation-Restoration"

  if tokenizer == None:
    tokenizer = AutoTokenizer.from_pretrained( PATH)
  
  token_data = tokenizer.encode(transcript_text)[:-1]

  MAX_LEN = 256

  batch_token = [token_data[i:i+MAX_LEN-1] + [1] for i in range(0 , len(token_data) , MAX_LEN-1) ]

  batch_token[-1] = batch_token[-1] + [0]*(MAX_LEN - len(batch_token[-1]))

  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  if model == None :
    model = AutoModelForSeq2SeqLM.from_pretrained( PATH )
  
  model.to(torch.device(device))

  ids = torch.tensor(batch_token).to(device, dtype = torch.long)
  batch_list = []
  batch_size = 16
  for i in range(0,len(batch_token),batch_size):
    batch_list.append(batch_token[i:i+batch_size])

  pred_batch = []
  for batch in batch_list:
    ids = torch.tensor(batch).to(device, dtype = torch.long)
    generated_ids = model.generate(
                    input_ids = ids,
                    max_length=256, 
                    num_beams=2,
                    repetition_penalty=2.5, 
                    length_penalty=1.0, 
                    early_stopping=True
                    )
    pred_batch.append(generated_ids)



  preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for batch in pred_batch for g in batch]

  output_text = " ".join(preds)

  return output_text

In [ ]:
def eval_test_dataset(dataset,model,tokenizer):

  perform_dict = {}
  for puct in ['.',',','?','"',';',':','-','!']:
    perform_dict[puct]= {'tr_matched':0,'tr_no_matched':0,'total_marks':0}


  tr_recall = 0.0
  tr_precisions = 0.0
  tr_f1_score = 0.0
  total_step = 0

  for text in tqdm(dataset):

    orig_text = text[0]
    ctext = text[1]
    pred_text = predict_full_text(ctext , model, tokenizer)
    pred_tokens = return_punctuation_gram(pred_text)
    label_tokens = return_punctuation_gram(orig_text)
    common_gram = set(label_tokens).intersection(pred_tokens)
    ep = 1e-5

    precisions = len(common_gram)/(len(pred_tokens)+ep)
    recall = len(common_gram)/(len(label_tokens)+ep)
    f1_score = 2*precisions*recall/(precisions+recall + ep)

    for gram in common_gram:
      puct_mark = gram.split(' ')[1]
      perform_dict[puct_mark]['tr_matched']+=1
      perform_dict[puct_mark]['total_marks']+=1

    diff = set(label_tokens).difference(common_gram)

    for gram in diff:
      puct_mark = gram.split(' ')[1]
      perform_dict[puct_mark]['tr_no_matched']+=1
      perform_dict[puct_mark]['total_marks']+=1
    
    tr_precisions += precisions
    tr_recall += recall
    tr_f1_score += f1_score
    total_step+=1

    result = {
      'precisions':tr_precisions/total_step,
      'recall': tr_recall/total_step,
      'F1_score': tr_f1_score/total_step,
      'Perform_metrics': perform_dict
  }

  return result

In [ ]:
test_processed_data = []
for text in test_data:
  ctext = re.sub('[\.,\?\";:\-!]',' ', text)      
  test_processed_data.append((text, ctext ))
  
result = eval_test_dataset(test_processed_data[:100], model , tokenizer)

100%|██████████| 100/100 [10:54<00:00,  6.54s/it]


In [ ]:
print("Test Dataset Evaluation Summary :- ")
print("--------------------------------")
print("Precisions :- ", round(result['precisions'],3))
print("--------------------------------")
print("Recall :- ", round(result['recall'],3))
print("--------------------------------")
print("F1-Score :- ", round(result['F1_score'],3))
print("--------------------------------")
print("| Punctuation marks|  Accuracy   |")
print("|------------------|-------------|")
for val in result['Perform_metrics']:
  acc = result['Perform_metrics'][val]['tr_matched']/(result['Perform_metrics'][val]['total_marks']+1e-6)
  print("| "+str(val)+'                |'+str(round(acc,3))+"        |")
  print("|------------------|-------------|")

Test Dataset Evaluation Summary :- 
--------------------------------
Precisions :-  0.887
--------------------------------
Recall :-  0.752
--------------------------------
F1-Score :-  0.814
--------------------------------
| Punctuation marks|  Accuracy   |
|------------------|-------------|
| .                |0.863        |
|------------------|-------------|
| ,                |0.683        |
|------------------|-------------|
| ?                |0.769        |
|------------------|-------------|
| "                |0.0        |
|------------------|-------------|
| ;                |0.068        |
|------------------|-------------|
| :                |0.708        |
|------------------|-------------|
| -                |0.0        |
|------------------|-------------|
| !                |0.06        |
|------------------|-------------|


In [ ]:
result['Perform_metrics']

{'!': {'total_marks': 50, 'tr_matched': 3, 'tr_no_matched': 47},
 '"': {'total_marks': 0, 'tr_matched': 0, 'tr_no_matched': 0},
 ',': {'total_marks': 11785, 'tr_matched': 8048, 'tr_no_matched': 3737},
 '-': {'total_marks': 12, 'tr_matched': 0, 'tr_no_matched': 12},
 '.': {'total_marks': 8914, 'tr_matched': 7693, 'tr_no_matched': 1221},
 ':': {'total_marks': 497, 'tr_matched': 352, 'tr_no_matched': 145},
 ';': {'total_marks': 44, 'tr_matched': 3, 'tr_no_matched': 41},
 '?': {'total_marks': 802, 'tr_matched': 617, 'tr_no_matched': 185}}

### Testing with youtube Video URL



In [6]:
def get_transcript_by_url(youtube_url):

  video_id = youtube_url.split('/')[-1]

  try:
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
  except:
    raise("Youtube URL is Incorrect or Video transcript not accessible")
  
  transcript_text = ''
  for temp in transcript_list:
    if 'text' in temp.keys():
      transcript_text = transcript_text + ' '+ temp['text']

  return transcript_text

In [9]:
import textwrap

URL = 'https://youtu.be/Q2NJ2SW6nqg'

text = get_transcript_by_url(URL)
print("\n\n")
print("-------------------------------------------------------------------------")
print("TEXT :- \n")
print("\n".join(textwrap.wrap(text)))
print("-------------------------------------------------------------------------")
puct_text = predict_full_text(text)
print("PUNCTUATION TEXT :- \n")
print("\n".join(textwrap.wrap(puct_text)))
print("-------------------------------------------------------------------------")




-------------------------------------------------------------------------
TEXT :- 

 hello everyone this is ashish and welcome back to the podcast we call
talk and fire today i'm having with me komal and we are going to have
a discussion on spoken english or speaking skills in general is it
right komal how are you doing these days yes i'm doing pretty good so
we at the end of the podcast we are going to bring up some of the
currently occurring topics and some of the questions that we have been
asked in the comment section right so we are going to discuss that
just like the podcast basically i was telling it to komal today too
that a podcast cannot go to like 100 podcast with each episode having
a specific topic we are going to run out of topics so basically over
time we'll have to make it generalize that it is just episode 200 or
300 hope we reach it okay we are going to make it back yeah and what
we are going to do like always komal will be doing the tough job and
i'll be doing the

Token indices sequence length is longer than the specified maximum sequence length for this model (16407 > 512). Running this sequence through the model will result in indexing errors


PUNCTUATION TEXT :- 

hello everyone, this is ashish and welcome back to the podcast we call
talk and fire today. i'm having with me komal, and we are going to
have a discussion on spoken english or speaking skills in general, is
it right? komal, how are you doing these days? yes, i'm doing pretty
good, so we, at the end of the podcast, we are going to bring up some
of the currently occurring topics and some of the questions that we
have been asked in the comment section, right? so we are going to
discuss that just like the podcast, basically i was telling it to
komal today too that a podcast cannot go to like 100 podcast, with
each episode having a specific topic, we are going to run out of
topics, so basically over time, we'll have to make it generalize that
it is just episode 200 or 300, hope we reach it, okay, we are going to
make it back, yeah, and what we are going to do, like always, komal
will be doing the tough job, and i'll be doing the easy job. kumales
kamal has gathered so



---

<center> <b> THE END </b> </center>

---

